## Pydantic Models
- https://pydantic-docs.helpmanual.io/usage/models/

In [1]:
from enum import Enum
from pydantic import BaseModel, Field

In [2]:
class User(BaseModel):
    id: int
    name = 'Jane Doe'

In [3]:
user = User(id='123')
user

User(id=123, name='Jane Doe')

In [4]:
user_x = User(id='123.45')

ValidationError: 1 validation error for User
id
  value is not a valid integer (type=type_error.integer)

In [5]:
user.name

'Jane Doe'

In [6]:
user.dict()

{'id': 123, 'name': 'Jane Doe'}

In [7]:
user.json()

'{"id": 123, "name": "Jane Doe"}'

In [8]:
user.name

'Jane Doe'

In [9]:
user_b = user.copy()

In [10]:
user_b == user

True

In [11]:
user.schema()['properties']

{'id': {'title': 'Id', 'type': 'integer'},
 'name': {'title': 'Name', 'default': 'Jane Doe', 'type': 'string'}}

In [12]:
user.__config__

__main__.Config

In [13]:
user.__fields__

{'id': ModelField(name='id', type=int, required=True),
 'name': ModelField(name='name', type=str, required=False, default='Jane Doe')}

In [14]:
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: float | None = None


class Bar(BaseModel):
    apple = 'x'
    banana = 'y'


class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]

In [15]:
BaseModel.mro

<function BaseModel.mro()>

In [16]:
a = Foo(count=3, size=23.4)

In [17]:
m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
print(m)

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]


In [18]:
m

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])

In [19]:
print(m.dict())

{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [20]:
a = Foo(count=3, size=23.4)

### Validation Errors

In [21]:
from pydantic import BaseModel, ValidationError, conint


class Location(BaseModel):
    lat = 0.1
    lng = 10.1

In [22]:
class Model(BaseModel):
    is_required: float
    gt_int: conint(ge=5, lt=42)
    list_of_ints: list[int] = None
    a_float: float = None
    recursive_model: Location = None

In [23]:
data = dict(
    is_required = "223.211",
    list_of_ints=['1', 2, 3],
    a_float=5.44,
    recursive_model={'lat': 4.2, 'lng': 5.4},
    gt_int=4,
)

In [24]:
try:
    Model(**data)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "gt_int"
    ],
    "msg": "ensure this value is greater than or equal to 5",
    "type": "value_error.number.not_ge",
    "ctx": {
      "limit_value": 5
    }
  }
]


## Parse

In [25]:
import pickle
from datetime import datetime
from pathlib import Path

from pydantic import BaseModel, ValidationError

In [26]:
 class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None

In [28]:
m = User.parse_obj({'id': 123, 'name': 'James'})
m

User(id=123, signup_ts=None, name='James')

m

In [29]:
intent_result={'clientSettingCode': 'IntentAgentConfiguration', 'clientSettingID': '98f04d1f-af5f-41fb-95cb-4cc5888b51a7', 'settingValue': [{'key': 'sentence_preprocess', 'value': [{'key': 'preproc', 'value': 'adp-e-chat-sage-preproc', 'version': 'latest', 'endpointType': 'preproc'}, {'key': 'ner', 'value': 'roll-chat-ner', 'version': 'latest', 'endpointType': 'ner'}], 'endpointType': 'pipeline'}, {'key': 'shallow_service', 'value': 'adp-e-chat-sage-intents-shallow', 'version': 'latest', 'endpointType': 'intent'}, {'key': 'sbert_service', 'value': 'adp-e-chat-sage-intents-vectors', 'version': 'latest', 'endpointType': 'intent'}, {'key': 'bert_service', 'value': 'adp-e-chat-sage-intents-bert', 'version': 'latest', 'endpointType': 'intent'}, {'key': 'smalltalk_service', 'value': 'adp-e-chat-sage-smalltalk-bert', 'version': 'latest', 'endpointType': 'smalltalk'}, {'key': 'sentiment_service', 'value': None, 'version': 'latest', 'endpointType': 'emotion'}, {'key': 'facts_service', 'value': 'roll-chat-facts', 'version': 'latest', 'endpointType': 'question'}, {'key': 'kbase_service', 'value': 'roll-chat-knowledge-base', 'version': 'latest', 'endpointType': 'question'}], 'tagText': None, 'contextCountryCode': 'US'}

In [31]:
intent_settings = intent_result['settingValue'] if intent_result['clientSettingCode']=="IntentAgentConfiguration" else {}
intent_settings

[{'key': 'sentence_preprocess',
  'value': [{'key': 'preproc',
    'value': 'adp-e-chat-sage-preproc',
    'version': 'latest',
    'endpointType': 'preproc'},
   {'key': 'ner',
    'value': 'roll-chat-ner',
    'version': 'latest',
    'endpointType': 'ner'}],
  'endpointType': 'pipeline'},
 {'key': 'shallow_service',
  'value': 'adp-e-chat-sage-intents-shallow',
  'version': 'latest',
  'endpointType': 'intent'},
 {'key': 'sbert_service',
  'value': 'adp-e-chat-sage-intents-vectors',
  'version': 'latest',
  'endpointType': 'intent'},
 {'key': 'bert_service',
  'value': 'adp-e-chat-sage-intents-bert',
  'version': 'latest',
  'endpointType': 'intent'},
 {'key': 'smalltalk_service',
  'value': 'adp-e-chat-sage-smalltalk-bert',
  'version': 'latest',
  'endpointType': 'smalltalk'},
 {'key': 'sentiment_service',
  'value': None,
  'version': 'latest',
  'endpointType': 'emotion'},
 {'key': 'facts_service',
  'value': 'roll-chat-facts',
  'version': 'latest',
  'endpointType': 'question'

In [33]:
class IntentConfig(BaseModel):
    key: str
    value: str
    version: str = 'latest'
    endpointType: str

In [36]:
intent_settings[-1]

{'key': 'kbase_service',
 'value': 'roll-chat-knowledge-base',
 'version': 'latest',
 'endpointType': 'question'}

In [37]:
conf_1 = IntentConfig.parse_obj(intent_settings[-1])

In [38]:
conf_1

IntentConfig(key='kbase_service', value='roll-chat-knowledge-base', version='latest', endpointType='question')

## Dynamic Create Model

In [39]:
from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model('DynamicFoobarModel', foo=(str, ...), bar=123)


class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

In [40]:
DynamicFoobarModel

pydantic.main.DynamicFoobarModel

In [41]:
from pydantic import BaseModel, create_model


class FooModel(BaseModel):
    foo: str
    bar: int = 123

In [42]:
BarModel = create_model(
    'BarModel',
    apple='russet',
    banana='yellow',
    __base__=FooModel,
)

In [43]:
print(BarModel)

<class 'pydantic.main.BarModel'>


In [46]:
BarModel.Config.fields

{}

In [47]:
print(BarModel.__fields__.keys())

dict_keys(['foo', 'bar', 'apple', 'banana'])


In [53]:
a = BarModel({})

TypeError: __init__() takes exactly 1 positional argument (2 given)

## Dataclass

In [54]:
from datetime import datetime
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    signup_ts: datetime = None

In [55]:
user_1 = User(id='42', signup_ts='2032-06-21T12:00')
user_1

User(id=42, name='John Doe', signup_ts=datetime.datetime(2032, 6, 21, 12, 0))

In [59]:
user_2 = User(id='43', signup_ts='2032-06-21T14:00')
user_2

User(id=43, name='John Doe', signup_ts=datetime.datetime(2032, 6, 21, 14, 0))

In [60]:
class FooModel(BaseModel):
    foo: str
    bar: int = 123
    
    Cla

In [61]:
user = FooModel(foo="testing", bar=1000)

In [63]:
user.json()

'{"foo": "testing", "bar": 1000}'

In [68]:
user.Config

{}